In [ ]:
import numpy as np
import pandas as pd

import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix
from PIL import Image
from matplotlib.pyplot import figure

import numpy as np
import pandas as pd

import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

import tensorflow as tf
from tensorflow import keras
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
print("TF version:", tf.__version__)

TF version: 2.15.0


In [ ]:
image_size = 224
batch_size = 16


target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)

In [ ]:
!kaggle datasets download -d tarundalal/dangerous-insects-dataset

 99% 441M/447M [00:03<00:00, 177MB/s]
100% 447M/447M [00:03<00:00, 145MB/s]


In [ ]:
!unzip /content/dangerous-insects-dataset.zip

Archive:  /content/dangerous-insects-dataset.zip
  inflating: farm_insects/Africanized Honey Bees (Killer Bees)/Image_1.jpg  
  inflating: farm_insects/Africanized Honey Bees (Killer Bees)/Image_10.jpg  
  inflating: farm_insects/Africanized Honey Bees (Killer Bees)/Image_100.jpg  
  inflating: farm_insects/Africanized Honey Bees (Killer Bees)/Image_102.jpg  
  inflating: farm_insects/Africanized Honey Bees (Killer Bees)/Image_104.jpg  
  inflating: farm_insects/Africanized Honey Bees (Killer Bees)/Image_105.jpg  
  inflating: farm_insects/Africanized Honey Bees (Killer Bees)/Image_106.jpg  
  inflating: farm_insects/Africanized Honey Bees (Killer Bees)/Image_107.png  
  inflating: farm_insects/Africanized Honey Bees (Killer Bees)/Image_108.jpg  
  inflating: farm_insects/Africanized Honey Bees (Killer Bees)/Image_11.jpg  
  inflating: farm_insects/Africanized Honey Bees (Killer Bees)/Image_110.jpg  
  inflating: farm_insects/Africanized Honey Bees (Killer Bees)/Image_116.jpg  
  infla

**Load and Transform Data**

In [ ]:
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMAGE_SHAPE = (224, 224)
train_datagen = ImageDataGenerator(rescale=1. / 255.,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        validation_split=0.15)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255., validation_split=0.15)
valid_generator = valid_datagen.flow_from_directory(
   '/content/farm_insects',
    subset="validation",
    batch_size = batch_size,
    target_size=IMAGE_SHAPE
)

train_generator = train_datagen.flow_from_directory(
        '/content/farm_insects',
        target_size=(224, 224),
        batch_size = batch_size)

Found 231 images belonging to 15 classes.
Found 1579 images belonging to 15 classes.


In [ ]:
model = tf.keras.models.Sequential([

    # The first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Flatten the results to feed into a dense layer
    tf.keras.layers.Flatten(),
    # 128 neuron in the fully-connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # 5 output neurons for 5 classes with the softmax activation
    tf.keras.layers.Dense(15, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 64)        0

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=1e-3)
from keras.metrics import Precision , Recall

model.compile(
  optimizer=optimizer,
  loss='categorical_crossentropy',
  metrics=['accuracy',Precision(),Recall()])

In [ ]:
# total_sample = train_generator.n
# n_epochs = 36
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('best_model.hdf5', monitor = 'val_accuracy', verbose = 1, save_best_only = True)

In [ ]:
history = model.fit(
        train_generator,
        epochs = 30,
        validation_data= valid_generator,
        callbacks = [checkpoint])

Epoch 1/30
99/99 [==============================] - ETA: 0s - loss: 0.7153 - accuracy: 0.7568 - precision: 0.8550 - recall: 0.6757
Epoch 1: val_accuracy improved from 0.93506 to 0.95238, saving model to best_model.hdf5
99/99 [==============================] - 106s 1s/step - loss: 0.7153 - accuracy: 0.7568 - precision: 0.8550 - recall: 0.6757 - val_loss: 0.2210 - val_accuracy: 0.9524 - val_precision: 0.9764 - val_recall: 0.8961
Epoch 2/30
99/99 [==============================] - ETA: 0s - loss: 0.7675 - accuracy: 0.7410 - precision: 0.8350 - recall: 0.6409
Epoch 2: val_accuracy did not improve from 0.95238
99/99 [==============================] - 106s 1s/step - loss: 0.7675 - accuracy: 0.7410 - precision: 0.8350 - recall: 0.6409 - val_loss: 0.2293 - val_accuracy: 0.9307 - val_precision: 0.9720 - val_recall: 0.9004
Epoch 3/30
99/99 [==============================] - ETA: 0s - loss: 0.7142 - accuracy: 0.7378 - precision: 0.8435 - recall: 0.6555
Epoch 3: val_accuracy did not improve from 0